# Gradient Boosting Model

The GradientBoostingClassifier is a machine learning model for recognizing face emotions. It is a member of the gradient boosting algorithm family and is noted for its ability to handle complex datasets and build high-performance models.




In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report

In [2]:
data = pd.read_csv('fer2013.csv')


In [3]:
# Spliting into train, val, test sets
train_data = data[data.Usage == 'Training']
test_data = data[data.Usage == 'PrivateTest']
val_data = data[data.Usage == 'PublicTest']

In [4]:
# Preprocessing images
def preprocess(data):
    X = []
    y = []
    for i in range(len(data)):
        img = data.iloc[i]['pixels'].split(' ')
        img = np.array(img, dtype='float32')
        img = img / 255.0
        X.append(img)
        y.append(data.iloc[i]['emotion'])
    X = np.array(X)
    y = np.array(y)
    return X, y

X_train, y_train = preprocess(train_data)
X_val, y_val = preprocess(val_data)
X_test, y_test = preprocess(test_data)

In [5]:
X_train = X_train.reshape(X_train.shape[0], -1)
X_val = X_val.reshape(X_val.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)



n_estimators: we can use this to specify the number of boosting stages or decision trees to create. In our situation, we set it to 200, which means 200 decision trees will be generated.

max_depth: The maximum depth of each decision tree is determined by this option. A larger value, such as 20, allows the trees to have more splits and capture more intricate data linkages. We fixed on this number after trying several depths.

In [6]:
clf = GradientBoostingClassifier(n_estimators=200, max_depth=20, learning_rate=0.1, validation_fraction=0.2, n_iter_no_change=10, verbose=1)


In [7]:
clf.fit(X_train, y_train)


      Iter       Train Loss   Remaining Time 
         1           1.3874         3305.64m
         2           1.1312         2802.75m
         3           0.9473         2631.97m
         4           0.8068         2538.11m
         5           0.6900         2484.26m
         6           0.5914         2476.54m
         7           0.5078         2427.32m
         8           0.4333         2387.64m
         9           0.3722         2352.74m
        10           0.3204         2322.87m
        20           0.0701         2090.24m
        30           0.0164         1959.93m


GradientBoostingClassifier(max_depth=20, n_estimators=200, n_iter_no_change=10,
                           validation_fraction=0.2, verbose=1)

In [8]:
y_pred = clf.predict(X_test)

In [9]:
y_train_pred = clf.predict(X_train)

In [12]:
from sklearn.metrics import classification_report

target_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
print('Training Classification')
print(classification_report(y_train, y_train_pred, target_names=target_names, labels=[0, 1, 2, 3, 4, 5, 6]))


Training Classification
              precision    recall  f1-score   support

       Angry       0.93      0.84      0.88      3995
     Disgust       0.81      0.84      0.83       436
        Fear       0.91      0.85      0.88      4097
       Happy       0.85      0.94      0.89      7215
         Sad       0.86      0.86      0.86      4830
    Surprise       0.95      0.90      0.92      3171
     Neutral       0.87      0.87      0.87      4965

    accuracy                           0.88     28709
   macro avg       0.88      0.87      0.88     28709
weighted avg       0.88      0.88      0.88     28709



In [13]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.43382557815547507
              precision    recall  f1-score   support

           0       0.42      0.25      0.32       491
           1       0.23      0.35      0.28        55
           2       0.41      0.25      0.31       528
           3       0.47      0.71      0.56       879
           4       0.31      0.32      0.31       594
           5       0.71      0.53      0.61       416
           6       0.39      0.39      0.39       626

    accuracy                           0.43      3589
   macro avg       0.42      0.40      0.40      3589
weighted avg       0.44      0.43      0.42      3589



The model's performance is average, with F1-scores ranging from 0.28 to 0.61 for various emotion categories. It is struggling to correctly predicting minority classes (Disgust and Fear) while performing well on majority classes (Happy, Surprise). The macro average F1-score is 0.40, indicating a moderate overall performance.

Below We are using a feature selection technique called the SelectFromModel class from scikit-learn. The underlying estimators for feature selection are the LassoCV and RidgeCV models.

In [14]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV, RidgeCV

In [15]:

selector = SelectFromModel(LassoCV())
selector = SelectFromModel(RidgeCV())

# creating pipeline with selector and GradientBoostingClassifier
clf = make_pipeline(selector, GradientBoostingClassifier(n_estimators=200, max_depth=20, learning_rate=0.1, validation_fraction=0.2, n_iter_no_change=10, verbose=1))

In [16]:
clf.fit(X_train, y_train)


      Iter       Train Loss   Remaining Time 
         1           1.3764          878.43m
         2           1.1188          901.41m
         3           0.9361          907.05m
         4           0.7881          908.13m
         5           0.6686          908.08m
         6           0.5712          903.18m
         7           0.4873          898.85m
         8           0.4172          895.48m
         9           0.3577          883.75m
        10           0.3065          856.54m
        20           0.0656          711.89m
        30           0.0154          640.53m


Pipeline(steps=[('selectfrommodel',
                 SelectFromModel(estimator=RidgeCV(alphas=array([ 0.1,  1. , 10. ])))),
                ('gradientboostingclassifier',
                 GradientBoostingClassifier(max_depth=20, n_estimators=200,
                                            n_iter_no_change=10,
                                            validation_fraction=0.2,
                                            verbose=1))])

In [17]:
y_pred = clf.predict(X_test)

In [18]:
y_train_pred = clf.predict(X_train)

In [19]:
from sklearn.metrics import classification_report

target_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
print('Training Classification')
print(classification_report(y_train, y_train_pred, target_names=target_names, labels=[0, 1, 2, 3, 4, 5, 6]))


Training Classification
              precision    recall  f1-score   support

       Angry       0.91      0.84      0.88      3995
     Disgust       0.80      0.86      0.83       436
        Fear       0.92      0.85      0.88      4097
       Happy       0.85      0.94      0.89      7215
         Sad       0.87      0.86      0.86      4830
    Surprise       0.94      0.90      0.92      3171
     Neutral       0.87      0.87      0.87      4965

    accuracy                           0.88     28709
   macro avg       0.88      0.88      0.88     28709
weighted avg       0.89      0.88      0.88     28709



In [20]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.42490944552800225
              precision    recall  f1-score   support

           0       0.37      0.24      0.29       491
           1       0.26      0.33      0.29        55
           2       0.40      0.24      0.30       528
           3       0.45      0.69      0.55       879
           4       0.31      0.31      0.31       594
           5       0.68      0.51      0.59       416
           6       0.40      0.41      0.40       626

    accuracy                           0.42      3589
   macro avg       0.41      0.39      0.39      3589
weighted avg       0.43      0.42      0.41      3589



The model performs to similar to previous models with slight dip , with F1-scores ranging from 0.29 to 0.59 for various emotion categories. It still struggles to forecast the minority classes (Disgust and Fear) while performing quite well on the majority classes (Happy, Surprise). The macro average F1-score stays at 0.39, indicating a moderate overall performance.

In [22]:
import joblib

joblib.dump(clf, "boosting_model.pkl")

['boosting_model.pkl']